In [1]:
from DemoServerDB import ConnectDatabase
from mapping import doc

In [2]:
temp = ConnectDatabase()

In [3]:
temp.conn

<connection object at 0x7f79a4ce2398; dsn: 'dbname=postgresdemotwo user=clientdemo password=xxxxxxxxxxx host=postgresdemotwo.c3yxphqgag3s.ap-southeast-1.rds.amazonaws.com port=5432 sslmode=require', closed: 0>

In [4]:
doc['column_map']['TRANSACTION_MASTER']

{'category': 'category',
 'cust_id': 'id',
 'product_id': 'prod_id',
 'quantity': 'purchasequantity',
 'timestamp': 'date'}

In [5]:
query = 'SELECT "%s","%s","%s" FROM "%s"'%(doc['column_map']['TRANSACTION_MASTER']['cust_id'],
                                           doc['column_map']['TRANSACTION_MASTER']['category'],
                                           doc['column_map']['TRANSACTION_MASTER']['timestamp'],
                                           doc['table_map']['TRANSACTION_MASTER'])

In [6]:
print query

SELECT "id","category","date" FROM "trans"


In [7]:
temp.conn.rollback()

In [8]:
temp.cur.execute(query)

In [9]:
col_names = [desc[0] for desc in temp.cur.description]

In [10]:
col_names

['id', 'category', 'date']

In [11]:
import pandas as pd
import numpy as np
import datetime as dt

In [12]:
trans_table = pd.DataFrame(temp.cur.fetchall(), columns=col_names)

In [13]:
trans_table.head()

,id,category,date
0,555549182,5613,2012-10-21
1,555549182,5307,2012-10-21
2,555549182,2108,2012-10-21
3,555549182,6315,2012-10-21
4,555549182,5827,2012-10-21


In [14]:
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['cust_id']:'cust_id'},inplace=True)
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['category']:'category'},inplace=True)
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['timestamp']:'timestamp'},inplace=True)

In [15]:
trans_table.head()

,cust_id,category,timestamp
0,555549182,5613,2012-10-21
1,555549182,5307,2012-10-21
2,555549182,2108,2012-10-21
3,555549182,6315,2012-10-21
4,555549182,5827,2012-10-21


In [16]:
trans_table['timestamp'] = pd.to_datetime(trans_table['timestamp'])

In [17]:
trans_table.shape

(71932603, 3)

In [ ]:
len(trans_table.cust_id.unique())

In [ ]:
len(trans_table.category.unique())

In [18]:
group_data = trans_table.groupby(['cust_id', 'category']).size().reset_index().rename(columns={0:'counter'})

In [19]:
group_data.head()

,cust_id,category,counter
0,100010021,201,1
1,100010021,311,1
2,100010021,410,1
3,100010021,416,8
4,100010021,417,1


In [20]:
group_data.shape

(11965917, 3)

In [32]:
repeated_data = group_data.query('counter != 1')

In [33]:
repeated_data.head()

,cust_id,category,counter
3,100010021,416,8
6,100010021,427,6
7,100010021,501,5
8,100010021,519,3
11,100010021,610,2


In [34]:
repeated_data.reset_index(drop=True, inplace=True)

In [35]:
repeated_data.head()

,cust_id,category,counter
0,100010021,416,8
1,100010021,427,6
2,100010021,501,5
3,100010021,519,3
4,100010021,610,2


In [23]:
len(repeated_data.cust_id.unique())

62276

In [24]:
len(repeated_data.category.unique())

828

In [36]:
repeated_data.head()

,cust_id,category,counter
0,100010021,416,8
1,100010021,427,6
2,100010021,501,5
3,100010021,519,3
4,100010021,610,2


In [37]:
trans_table.head()

,cust_id,category,timestamp
0,555549182,5613,2012-10-21
1,555549182,5307,2012-10-21
2,555549182,2108,2012-10-21
3,555549182,6315,2012-10-21
4,555549182,5827,2012-10-21


In [38]:
xyz = pd.merge(trans_table, repeated_data, on=['cust_id', 'category'], how='left')

In [39]:
xyz.head()

,cust_id,category,timestamp,counter
0,555549182,5613,2012-10-21,18.0
1,555549182,5307,2012-10-21,18.0
2,555549182,2108,2012-10-21,25.0
3,555549182,6315,2012-10-21,95.0
4,555549182,5827,2012-10-21,19.0


In [40]:
xyz.shape

(71932603, 4)

In [41]:
result = xyz.groupby(['cust_id', 'category'])['timestamp'].agg(['max', 'min']).reset_index()

In [42]:
result.head()

,cust_id,category,max,min
0,100010021,201,2013-05-12,2013-05-12
1,100010021,311,2013-05-12,2013-05-12
2,100010021,410,2013-02-25,2013-02-25
3,100010021,416,2012-11-05,2012-06-01
4,100010021,417,2012-10-24,2012-10-24


In [43]:
result['time'] = result['max']-result['min']

In [44]:
xyz2 = pd.merge(xyz, result, on=['cust_id', 'category'], how='left')

In [45]:
xyz2.head()

,cust_id,category,timestamp,counter,max,min,time
0,555549182,5613,2012-10-21,18.0,2013-06-17,2012-03-22,452 days
1,555549182,5307,2012-10-21,18.0,2013-06-17,2012-03-05,469 days
2,555549182,2108,2012-10-21,25.0,2012-11-19,2012-04-16,217 days
3,555549182,6315,2012-10-21,95.0,2013-06-17,2012-03-05,469 days
4,555549182,5827,2012-10-21,19.0,2013-06-03,2012-04-30,399 days


In [46]:
len(xyz2.cust_id.unique())

62308

In [47]:
len(xyz2.category.unique())

831

In [48]:
cust_repur = xyz2

In [49]:
cust_repur.head()

,cust_id,category,timestamp,counter,max,min,time
0,555549182,5613,2012-10-21,18.0,2013-06-17,2012-03-22,452 days
1,555549182,5307,2012-10-21,18.0,2013-06-17,2012-03-05,469 days
2,555549182,2108,2012-10-21,25.0,2012-11-19,2012-04-16,217 days
3,555549182,6315,2012-10-21,95.0,2013-06-17,2012-03-05,469 days
4,555549182,5827,2012-10-21,19.0,2013-06-03,2012-04-30,399 days


In [50]:
cust_repur.shape

(71932603, 7)

In [51]:
cust_repur.reset_index(drop=True, inplace=True)

In [52]:
cust_repur.head()

,cust_id,category,timestamp,counter,max,min,time
0,555549182,5613,2012-10-21,18.0,2013-06-17,2012-03-22,452 days
1,555549182,5307,2012-10-21,18.0,2013-06-17,2012-03-05,469 days
2,555549182,2108,2012-10-21,25.0,2012-11-19,2012-04-16,217 days
3,555549182,6315,2012-10-21,95.0,2013-06-17,2012-03-05,469 days
4,555549182,5827,2012-10-21,19.0,2013-06-03,2012-04-30,399 days


In [ ]:
list_of_indexes = [np.argmax(g['timestamp']) for l, g in cust_repur.groupby(['cust_id','category'])]

In [ ]:
cust_repur_original = cust_repur.ix[list_of_indexes]

In [ ]:
###cust_repur_original = cust_repur

In [ ]:
cust_repur_original.head()

In [ ]:
cust_repur_original.reset_index(drop=True,inplace=True)

In [ ]:
cust_repur_original.head()

In [ ]:
cust_repur_original['days_int'] = cust_repur_original['time'].dt.days

In [ ]:
cust_repur_original.head()

In [ ]:
cust_repur_original['repurchase_time'] = (cust_repur_original['days_int']-1) / (cust_repur_original['counter']-1)

In [ ]:
cust_repur_original.head()

In [ ]:
len(cust_repur_original.category.unique())

In [ ]:
abc = pd.pivot_table(cust_repur_original, index=['cust_id'], columns=['category'], values=['repurchase_time'], aggfunc='first')

In [ ]:
abc.shape

In [ ]:
abc.head()

In [ ]:
type(abc)

In [ ]:
mi = abc.columns

In [ ]:
mii = mi.tolist()

In [ ]:
ind = pd.Index([e[1] for e in mii])

In [ ]:
abc.columns = ind

In [ ]:
abc.head()

In [ ]:
categories = pd.DataFrame(abc.columns)

In [ ]:
categories.rename(columns={0:'category'}, inplace=True)

In [ ]:
art = abc[categories.category[categories.index]].sum() / abc[categories.category[categories.index]].count()

In [ ]:
type(art)

In [ ]:
art = art.to_frame()

In [ ]:
art.reset_index(inplace=True)

In [ ]:
art.rename(columns={'index':'category'}, inplace=True)

In [ ]:
art.rename(columns={0:'average_repurchase_time'}, inplace=True)

In [ ]:
art.head()

In [ ]:
cust_repur_original.head()

In [ ]:
cust_repur_original['time_diff'] = dt.date.today() - cust_repur_original["timestamp"].dt.date

In [ ]:
cust_repur_original.head()

In [ ]:
cust_repur_original.tail()

In [ ]:
cust_repur_original.counter = cust_repur_original.counter.fillna(0)

In [ ]:
cust_repur_original.head()

In [ ]:
cust_repur_original.head()

In [ ]:
cust_repur_original.tail()

In [ ]:
cust_repur_original['time_diff_int'] = cust_repur_original['time_diff'].dt.days

In [ ]:
cust_repur_original.head()

In [ ]:
cust_repur_original_final = cust_repur_original.merge(art, on=['category'], how='left')

In [ ]:
cust_repur_original_final.head()

In [ ]:
cust_repur_original_final.repurchase_time.fillna(cust_repur_original_final.average_repurchase_time, inplace=True)

In [ ]:
cust_repur_original_final.head()

In [ ]:
cust_repur_original_final['depriotarize'] = np.where(cust_repur_original_final['time_diff_int'] > cust_repur_original_final['repurchase_time'], 'Recommend', 'Do_not_recommend')

In [ ]:
cust_repur_original.head()

In [ ]:
del cust_repur_original['timestamp']

In [ ]:
del cust_repur_original['counter']

In [ ]:
del cust_repur_original['time']

In [ ]:
del cust_repur_original['days_int']

In [ ]:
del cust_repur_original['repurchase_time']

In [ ]:
del cust_repur_original['time_diff']

In [ ]:
del cust_repur_original['time_diff_int']

In [ ]:
cust_repur_original.head()